In [51]:
%load_ext autoreload


In [79]:
from random import random, randint
import numpy as np

In [144]:
nlines = 31
ncols = 7
k = 8
def checa_vazio(*m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1

def cria_novo_matrix(nlines, ncols, k):
    m=np.zeros((nlines, ncols))
    for i in range(ncols):
      ltot = 0
      while ltot < k:
         n = randint(0,nlines-1)
         if m[n][i] != 1:
               m[n][i] = 1
               ltot += 1
    index = checa_vazio(*m)
    while index != -1:
        n = randint(0,ncols-1)
        m[index][n] = 1
        s = -1
        ## atualizar matrix pois agora tem mais de k 1's na coluna
        k = randint(0, nlines-1)
        while m[k][n]== 0:
            k = randint(0, nlines-1)
        m[k][n] = 0
        index = checa_vazio(*m)
    return m 


In [172]:
nlines = 31
ncols = 7
k = 8
#checa se alguem foi usado mais que k vezes
def checa_k(*m):
    i=0
    j=0
    for i in range(len(m[0])):
        t=0
        for j in range(len(m)):
            t+=m[j][i]
            # j+=1
        if t>k:
            return i #retorna indice do que apareceu mais de k vezes
        # i+=1
    return -1

def checa_vazio(*m):    
    for i in range(len(m)):
        sum = m[i].sum()
        if sum == 0:
            return i
    return -1
    
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[line][col] != y[line][col]:
                tuplas.append((line, col))

    casas_sorteadas = []
    n = min(4, len(tuplas))
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(num)
    
    ## np.array
    childs = [x, y]
    bits = np.zeros(n)
    bits[0] = 1
    m = deepcopy(x)
    l = pow(2,n)
    for i in range(l-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[a][b]
        um = np.zeros(n)
        um[0] = 1
        bits = soma_bits(bits, um, n)
        if checa_k(*matrix) == -1 and checa_vazio(*matrix) == -1:
            childs.append(matrix)
    return childs

In [168]:

etapa_dif = []
for i in range(1,32):
    etapa_dif.append(10*i)
personagem_agilidade = [1.8, 1.6, 1.6, 1.6, 1.4, 0.9, 0.7]
nlines = 31
ncols = 7
k = 8
# custo do tempo a cada etapa: dificuldade/ agilidade na etapa
def custo_tempo(etapa_dif, personagem_agilidade, matrix_genetica):
    matrix = np.divide(etapa_dif, np.matmul(matrix_genetica, personagem_agilidade))
    return np.sum(matrix)



def busca(p, lista): # returna indice i em que lista[i-1] < p < lista[i]
	low = 0
	high = len(lista) - 1
	while low <= high:
		middle = low + (high - low) // 2
		if lista[middle] >= p and lista[middle-1] < p: # lista[middle-1] < p <= lista[middle]
			return middle
		elif lista[middle] <= p:
			low = middle + 1 
		else:
			high = middle - 1
	return 0

def random_selection(population):
    times = [custo_tempo(etapa_dif, personagem_agilidade, person) for person in population]
    times = np.divide(1, times) 
    probs = np.divide(times, np.sum(times)) ## mais provavel, menor tempos
    lista = []
    psum = 0
    n= len(probs)
    for i in range(n):
        psum += probs[i]
        lista.append(psum)
    p = random()
    j = busca(p, lista)
    return population[j] 
def best_individual(population):
    times = [custo_tempo(etapa_dif, personagem_agilidade, person) for person in population]    
    best_time = times[0]
    index = 0
    for i in range(1, len(times)):
        if times[i] < best_time:
            best_time = times[i]
            index = i
    return best_time, population[index]

def die(population):
    ## population (times, pop)
    tam = len(population)
    excess = 2000 - tam
    # tupla (tempo, child)
    times = [(custo_tempo(etapa_dif, personagem_agilidade, person), person) for person in population]
    times.sort(key=lambda y: y[0])
    for i in range(excess):
        times.pop(tam-i)
    population = []
    for i in range(excess):
        population.append(times[i][1])
    return population



def genetic_algorithm(population,n): ## population list of matrices
    best_time, fit_ind = best_individual(population)
    print('best_time', best_time)

    for j in range(n):
        ## tamanho pop
        # ideia tirar caras ruins
            new_population = []
            # evaluate best individual...
            for i in range(len(population)):
                x = random_selection(population)
                y = random_selection(population)
                while np.array_equal(x,y):
                    y = random_selection(population)
                childs = reproduce1(x,y)
                for child in childs:
                    new_population.append(child)
                # p = random()
                # if (p < 0.01):
                #     child = mutate(child)
            print('new pop', len(new_population))
            population = new_population
            time, ind = best_individual(population)
            if time < best_time:
                best_time = time
                fit_ind = ind
                print('time', time)
                print('best_time', best_time)
            if len(new_population) < 2000:
                new_population = die(new_population)
    return fit_ind

In [173]:
population = list()
for i in range(100):
    population.append(cria_novo_matrix(nlines, ncols, k))

In [174]:
genetic_algorithm(population,10)

best_time 2278.687997494435
new pop 492
new pop 2684
time 2236.8654017894864
best_time 2236.8654017894864
new pop 16670
time 2224.3665689230065
best_time 2224.3665689230065


In [159]:
min(4,1)

1

In [148]:
x = population[0]
y= population[1]

In [134]:
x

array([[1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [135]:
y

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 1., 0.],
       [0., 0., 1., 1.]])

In [140]:
childs= reproduce1(x,y,2)

In [141]:
len(childs)

3

In [143]:
np.array_equal(childs[1], childs[2])

False

In [68]:
a = genetic_algorithm(population, 40)

best_time 2452.0840004074826
new pop 0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys


IndexError: list index out of range

In [ ]:

def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits

# assume que soma resultado vai a no maximo n '1s
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

def reproduce1(x,y): 
    tuplas = []
    for line in range(nlines):
        for col in range(ncols):
            if x[line][col] != y[line][col]:
                tuplas.append((line, col))

    casas_sorteadas = []
    n=4
    for i in range(n):
        num = randint(0, len(tuplas)-1)
        casas_sorteadas.append(tuplas[num])
        tuplas.pop(k)
    
    ## np.array
    childs = [x, y]
    bits = np.array([0,0,0,0])
    m = deepcopy(x)
    n = pow(2,n)
    for i in range(n-1): ## criancas podem ser x e y
        matrix = deepcopy(m)
        for j in range(n):
            # percorrer bits
            if bits[j] == 1:
                a = casas_sorteadas[j][0]
                b = casas_sorteadas[j][1]
                matrix[a][b] = y[a][b]
        um = np.array([1,0,0,0])
        bits = soma_bits(bits, um, n)
        if checa_k(matrix, k) == -1 and checa_vazio(matrix) == -1:
            childs.append(matrix)
    return childs

In [15]:
from random import random

In [33]:
m = np.matrix([[0,1], [1,1], [0,0]])
checa_vazio(*m)

soma 1
soma 2
soma 0


2

In [1]:
lista =[(1,2), (3,3)]

In [84]:
n = np.zeros((4, 5))

In [2]:
lista

[(1, 2), (3, 3)]

In [3]:
lista.append((3,4))

In [4]:
lista

[(1, 2), (3, 3), (3, 4)]

In [53]:
m = [[1,2],[3,4]]
n = deepcopy(m)

lista = []
lista.append(n)
m[0][0] = 7


In [54]:
n = deepcopy(m)
lista.append(n)

n[1][1] = 8

In [34]:
from copy import deepcopy

In [55]:
lista

[[[1, 2], [3, 4]], [[7, 2], [3, 8]]]

In [63]:
def bits(n):
    bits = [] 
    while n > 0:
        r = n%2
        n = n//2
        bits.append(r)
    return bits


In [64]:
bits(16)

[0, 0, 0, 0, 1]

In [75]:
def soma_bits(a,b, n ):
    r = a+b
    for i in range(n-1):
        if r[i] == 2:
            r[i] = 0
            r[i+1] +=1
    return r

In [76]:
soma_bits(a,b,3)

array([0, 1, 2])

In [73]:
a = np.array([1, 0, 1])
b = np.array([1,0,1])

In [68]:
lista + list1

array([1, 1, 2])

In [57]:
lista.pop(1)

[[7, 2], [3, 8]]

In [58]:
lista

[[[1, 2], [3, 4]]]

In [66]:
np.append(m, [2])

array([2.])

In [6]:
import numpy as np

In [7]:
m = np.zeros((3,3))

In [14]:
m

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [12]:
n

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [11]:
n[0][0] =1

In [9]:
n = m.copy()

In [47]:
lista = [[0,2], [1,1]]

In [50]:
lista[0][1]

2

In [51]:
lista[:,2]

TypeError: list indices must be integers or slices, not tuple